# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [64]:
# initial imports
import os
import pandas as pd
import panel as pn
pn.extension('plotly') 
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

# Initialize the Panel Extensions (for Plotly)


In [65]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX")

# Import Data

In [3]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [190]:
# Define Panel Visualization Functions
def housing_units_per_year_plot():
    """Housing Units Per Year."""   
    
    housing_units = sfo_data[["housing_units"]]
    housing_units_by_year = housing_units.groupby("year").mean().reset_index()

    fig_housing_units = housing_units_by_year.hvplot.bar(x="year", xlabel = "Year", y="housing_units", ylabel = "Housing Units", legend=False, title="Housing Units in San Francisco From 2010 to 2016").opts(ylim=(300000,390000))

    return fig_housing_units

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    gross_rent = sfo_data[["sale_price_sqr_foot", "gross_rent"]]
    avg_rent_by_year = gross_rent.groupby("year").mean()

    gross_rent = sfo_data[["gross_rent"]]
    avg_rent_by_year = gross_rent.groupby("year").mean().reset_index()

    avg_rent = avg_rent_by_year.plot.line(x = "year", y = "gross_rent", legend=False, title="Average Gross Rent in San Francisco")
    
    return avg_rent_by_year.plot.line(x = "year", y = "gross_rent", title="Average Gross Rent in San Francisco")

def average_sales_price():
    """Average Sales Price Per Year."""
    
    gross_sales = sfo_data[["sale_price_sqr_foot"]]
    avg_sales_by_year = gross_sales.groupby("year").mean().reset_index()

    avg_sales = avg_sales_by_year.hvplot.line(x = "year", y = "sale_price_sqr_foot", legend=False, title="Average Sales Price per Square Foot in San Francisco")

    return avg_sales

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    neighborhood = sfo_data[["neighborhood", "sale_price_sqr_foot"]]
    neighborhoods = round(neighborhood.groupby(["neighborhood"]).mean(),2) #.reset_index()
    
    return neighborhoods

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    neighborhood = sfo_data[["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"]]
    neighborhoods = round(neighborhood.groupby(["neighborhood"]).mean(),2)
    neighborhood_10 = neighborhoods.sort_values('sale_price_sqr_foot', ascending=False).head(10) #.reset_index()
    neighborhood_10

    return top_most_expensive_neighborhoods
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    neighborhood = sfo_data[["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"]]
    neighborhoods_10 = neighborhood.groupby(["neighborhood"]).mean()
    neighborhood_10 = neighborhoods_10.sort_values('sale_price_sqr_foot', ascending=False).head(10).reset_index()
    
    parallel_coordinates = px.parallel_coordinates(neighborhood_10, color="sale_price_sqr_foot")
       
    parallel_coordinates_pane  = pn.pane.Plotly(parallel_coordinates)
    return parallel_coordinates_pane

def parallel_categories():
    """Parallel Categories Plot."""
    
    neighborhood = sfo_data[["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"]]
    neighborhoods = neighborhood.groupby(["neighborhood"]).mean()
    neighborhood_10 = neighborhoods.sort_values('sale_price_sqr_foot', ascending=False).head(10).reset_index()
   
    parallel_categories = px.parallel_categories(
        neighborhood_10,
        dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        labels={
            "neighborhood": "Neighborhood",
            "sale_price_sqr_foot": "Sale Price per Square Foot",
            "housing_units": "Housing Units",
            "gross_rent": "Gross Rent"
        },
    )
    
    parallel_categories_pane  = pn.pane.Plotly(parallel_categories)
    return parallel_categories_pane
        
def neighborhood_map():
    """Neighborhood Map"""
    
# Calculate the mean values for each neighborhood

    neighborhood_values = sfo_data[["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"]]
    avg_neighborhood_values = neighborhood_values.groupby("neighborhood").mean().reset_index()
    
# Joined the average values with the neighborhood locations

    avg_neighborhood_values_cnt = (
        pd.concat([df_neighborhood_locations, avg_neighborhood_values], axis=1).dropna().drop(["neighborhood"], axis = 1)
    )  
    
# Read the Mapbox API key
    load_dotenv()
    map_box_api = os.getenv("mapbox")

# Set the Mapbox API
    px.set_mapbox_access_token(map_box_api)

    map_1 = px.scatter_mapbox( 
        avg_neighborhood_values_cnt, 
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        color="gross_rent",
        color_continuous_scale=px.colors.cyclical.IceFire,
        title="Average Sales Price per Square Foot and Gross Rent in San Francisco",
        size_max=15,
        zoom=10
    )
    
    map_pane  = pn.pane.Plotly(map_1)
    return map_pane

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [207]:
# Create dashboard tabs

row_of_parallel = pn.Row(parallel_categories, parallel_coordinates)
row = pn.Row(fig_housing_units, avg_rent)
row2 = pn.Row(neighborhoods, avg_sales)
row3 = pn.Row(neighborhood_10, neighborhood_map())
neighborhood_tab = pn.Row(neighborhoods, neighborhood_10)

plots_as_column = pn.Column(
    "# San Francisco Real Estate Rental Dashboard", row, row2, row3, row_of_parallel
)

tabs = pn.Tabs(
    ("Dashboard", plots_as_column),
    ("Neighborhoods", neighborhood_tab),
    ("Map Plot", neighborhood_map())
)
tabs

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Bars)
            [1] HoloViews(Curve)
        [2] Row
            [0] DataFrame(DataFrame)
            [1] HoloViews(Curve)
        [3] Row
            [0] DataFrame(DataFrame)
            [1] Plotly(Figure)
        [4] Row
            [0] Column
                [0] Column()
                [1] Row
                    [0] Plotly(Figure)
            [1] Column
                [0] Column()
                [1] Row
                    [0] Plotly(Figure)
    [1] Row
        [0] DataFrame(DataFrame)
        [1] DataFrame(DataFrame)
    [2] Plotly(Figure)

## Serve the Panel Dashboard

In [208]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")

# Set token using Plotly Express set function
px.set_mapbox_access_token(map_box_api)

In [209]:
tabs.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Bars)
            [1] HoloViews(Curve)
        [2] Row
            [0] DataFrame(DataFrame)
            [1] HoloViews(Curve)
        [3] Row
            [0] DataFrame(DataFrame)
            [1] Plotly(Figure)
        [4] Row
            [0] Column
                [0] Column()
                [1] Row
                    [0] Plotly(Figure)
            [1] Column
                [0] Column()
                [1] Row
                    [0] Plotly(Figure)
    [1] Row
        [0] DataFrame(DataFrame)
        [1] DataFrame(DataFrame)
    [2] Plotly(Figure)